In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
import glob


### Step 1. Feature eng

1. 새롭게 만든 파생변수가 논리적이어야 한다.
    - 1.1. Data driven
        - 만든 파생변수가 대출 승인 받았을때와 받지 않았을때의 distribution 차이. plot, SKlearn 
        - EDA 과정 다음 Link 확인 : https://dacon.io/competitions/official/235927/codeshare/6052?page=1&dtype=recent
        - EDA 과정(파생변수 만들기) : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent 
        
    - 1.2. Stat driven
        - 만든 파생변수가 통계적으로 is_applied(Y/N)에서 차이가 있는지 검정.
        - Numeric 한 변수면 -> 만든 파생변수가 정규분포를 따르는지 확인 샤피로 웰콘슨 테스트(정규분포 테스트) -> t-test
        - Numeric 하지 않으면 -> 비모수검정, 맨 흴트이 윌콕슨 테스트 등
        
2. 만든 파생변수 N개를 가지고 간단한 베이스라인 ML 모델로 확인 : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent
    - Step 1. 기본 변수로만 사용해서 성능확인 // 파생변수 N개만 사용해서 성능확인 -> 각각 2개확인
    - Step 2. 기본 변수로만 사용해서 성능확인 + 파생변수 1개씩 성능확인 (총 nCk 조합)
    
    
### Q1. Loan_limit, loan_rate가 중요한가?

- 대출이 승인되었는지 안되었는지를 예측하기 위해서 loan_limit & roan_rate가 중요한지 고민해보기
- "우선은 중요하지 않다" 라고 가정하고 결측치는 제거
- 대체적으로 is_applied가 1로 나타나는데 이는 label noise인가? 만약 label noise라면 확인할방법은?


## Label encoding 

1. income_type : `['EARNEDINCOME' 'EARNEDINCOME2' 'FREELANCER' 'OTHERINCOME' 'PRACTITIONER' 'PRIVATEBUSINESS']`

2. employment_type : `['계약직' '기타' '일용직' '정규직']`

3. houseown_type : `['기타가족소유' '배우자' '자가' '전월세']`

4. purpose : `['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']`



In [2]:
train = pd.read_csv("./prepro_data/train.csv")
val = pd.read_csv("./prepro_data/val.csv")
test = pd.read_csv("./prepro_data/test.csv")

In [3]:
train

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt
0,1435410,2022-05-20 07:57:59,35,267,55000000.0,13.3,0.0,665285,1972.0,0.0,...,760.0,55000000.0,0,296,3,0,10000000.0,3,5.0,32000000.0
1,1770977,2022-06-15 15:36:05,27,148,15000000.0,6.7,0.0,852126,1992.0,0.0,...,600.0,25000000.0,0,3,3,3,30000000.0,1,6.0,54000000.0
2,1948068,2022-05-09 09:19:11,50,142,9000000.0,15.9,0.0,69308,1977.0,1.0,...,670.0,36000000.0,0,399,3,2,10000000.0,3,9.0,85000000.0
3,1818991,2022-06-29 16:03:55,3,205,35000000.0,18.9,0.0,822770,1989.0,1.0,...,800.0,30000000.0,0,4,3,3,15000000.0,3,0.0,0.0
4,51114,2022-06-02 17:51:01,54,259,70000000.0,6.7,0.0,87323,1977.0,0.0,...,920.0,54000000.0,0,394,3,3,40000000.0,1,3.0,40000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439804,1442296,2022-03-15 14:24:34,10,149,34000000.0,12.2,0.0,224104,1989.0,0.0,...,980.0,37000000.0,0,194,3,3,500000000.0,6,0.0,0.0
8439805,1146485,2022-04-17 11:25:11,12,35,3000000.0,10.5,0.0,458140,1988.0,1.0,...,920.0,34000000.0,0,2,3,2,20000000.0,3,9.0,340000000.0
8439806,251466,2022-03-19 13:08:15,27,148,14000000.0,7.3,0.0,771371,1993.0,1.0,...,930.0,24000000.0,0,101,3,0,65000000.0,2,1.0,5000000.0
8439807,37188,2022-04-05 16:00:05,50,142,11000000.0,14.8,0.0,805543,1989.0,1.0,...,670.0,24000000.0,0,94,0,3,45000000.0,0,4.0,62000000.0
